In [5]:
import pandas as pd
import numpy as np
import yfinance as yf

In [330]:
INIT_DATE = "2016-12-30"
FINISH_DATE = "2024-10-24"

## Import Data

In [373]:
import pandas_market_calendars as mcal

# Create a calendar
calendar = mcal.get_calendar('CME_Currency')
calendar = calendar.date_range_htf(start=INIT_DATE, end=FINISH_DATE, frequency="1D")

In [374]:
def is_non_business_day(date, calendar):
    if date not in calendar:
        return 1
    return 0

In [375]:
is_non_business_day("2017-01-06", calendar)

0

In [376]:
is_non_business_day("2017-01-07", calendar)

1

### Exchange Rate

In [335]:
df_exchange_rate = pd.read_csv("../data/raw/exchangeRateIATA.csv", sep=";")
df_exchange_rate["dummy_non_bussiness_day"] = df_exchange_rate["dates"].apply(lambda x: is_non_business_day(x, calendar))
df_exchange_rate.head()

,dates,iata,dummy_non_bussiness_day
0,2017-01-01,672,1
1,2017-01-02,672,1
2,2017-01-03,670,0
3,2017-01-04,675,0
4,2017-01-05,674,0


### Copper

In [336]:
ticker_symbol = "HG=F"

ticker = yf.Ticker(ticker_symbol)
df_copper = ticker.history(start=INIT_DATE, end=FINISH_DATE, interval="1d")[["Close"]]
df_copper.reset_index(inplace=True)
df_copper.columns = ["dates", "copper_close"]
df_copper["dates"] = df_copper["dates"].dt.strftime("%Y-%m-%d")
df_copper["dates"] = pd.to_datetime(df_copper["dates"])

In [337]:
df_copper.head()

,dates,copper_close
0,2016-12-30,2.4975
1,2017-01-03,2.4810
2,2017-01-04,2.5480
3,2017-01-05,2.5295
4,2017-01-06,2.5380


### Dataframe

In [338]:
#https://go.updates.iata.org/exchange-rates-file-comparison?_gl=1*104hnoi*_gcl_au*ODUwOTc4MjAwLjE3MzczODE2Njg.*_ga*MTExNzgyMDgzNS4xNzM3MzgxNjY4*_ga_PLLG1EY0X0*MTczNzM4MTY2OC4xLjEuMTczNzM4MTY4Mi40Ni4wLjA.&_ga=2.3139890.306475781.1737381668-1117820835.1737381668
df_exchange_rate["iata_adj"] = df_exchange_rate["iata"].shift(-1)

# Cast date type
df_exchange_rate["dates"] = pd.to_datetime(df_exchange_rate["dates"])

# 
df_exchange_rate["y"] = np.log(df_exchange_rate["iata_adj"]) - np.log(df_exchange_rate["iata_adj"].shift())

# Lags
df_exchange_rate["usd_lag_1"] = df_exchange_rate["y"].shift(1)
df_exchange_rate["usd_lag_2"] = df_exchange_rate["y"].shift(2)
df_exchange_rate["usd_lag_sum_1_2"] = df_exchange_rate["usd_lag_1"] + df_exchange_rate["usd_lag_2"]

er_columns = ["dates", "dummy_non_bussiness_day", "iata_adj", "y", "usd_lag_1", "usd_lag_2", "usd_lag_sum_1_2"]
df_exchange_rate = df_exchange_rate[er_columns]
df_exchange_rate.dropna(inplace=True)

In [339]:
df = df_exchange_rate.merge(df_copper, how="left", on="dates")

In [340]:
df.head()

,dates,dummy_non_bussiness_day,iata_adj,y,usd_lag_1,usd_lag_2,usd_lag_sum_1_2,copper_close
0,2017-01-04,0,674.0,-0.001483,0.007435,-0.002981,0.004454,2.5480
1,2017-01-05,0,675.0,0.001483,-0.001483,0.007435,0.005952,2.5295
2,2017-01-06,0,668.0,-0.010425,0.001483,-0.001483,0.000000,2.5380
3,2017-01-07,1,668.0,0.000000,-0.010425,0.001483,-0.008942,NaN
4,2017-01-08,1,668.0,0.000000,0.000000,-0.010425,-0.010425,NaN


In [341]:
df["copper_close"] = df["copper_close"].ffill()

In [342]:
df.tail(10)

,dates,dummy_non_bussiness_day,iata_adj,y,usd_lag_1,usd_lag_2,usd_lag_sum_1_2,copper_close
2840,2024-10-14,0,927.0,-0.005379,0.000000,0.000000,0.000000,4.3635
2841,2024-10-15,0,928.0,0.001078,-0.005379,0.000000,-0.005379,4.3005
2842,2024-10-16,0,933.0,0.005373,0.001078,-0.005379,-0.004301,4.3315
2843,2024-10-17,0,940.0,0.007475,0.005373,0.001078,0.006452,4.2865
2844,2024-10-18,0,942.0,0.002125,0.007475,0.005373,0.012848,4.3460
2845,2024-10-19,1,942.0,0.000000,0.002125,0.007475,0.009600,4.3460
2846,2024-10-20,1,942.0,0.000000,0.000000,0.002125,0.002125,4.3460
2847,2024-10-21,0,954.0,0.012658,0.000000,0.000000,0.000000,4.3230
2848,2024-10-22,0,954.0,0.000000,0.012658,0.000000,0.012658,4.3480
2849,2024-10-23,0,951.0,-0.003150,0.000000,0.012658,0.012658,4.3045


In [343]:
df["copper_diff"] = np.log(df["copper_close"]) - np.log(df["copper_close"].shift())

df["copper_lag_1"] = df["copper_diff"].shift(1)
df["copper_lag_2"] = df["copper_diff"].shift(2)
df["copper_lag_3"] = df["copper_diff"].shift(3)
df["copper_lag_4"] = df["copper_diff"].shift(4)
df["copper_lag_sum_1_2"] = df["copper_lag_1"] + df["copper_lag_2"]

In [344]:
df.head(14)

,dates,dummy_non_bussiness_day,iata_adj,y,usd_lag_1,usd_lag_2,usd_lag_sum_1_2,copper_close,copper_diff,copper_lag_1,copper_lag_2,copper_lag_3,copper_lag_4,copper_lag_sum_1_2
0,2017-01-04,0,674.0,-0.001483,0.007435,-0.002981,0.004454,2.5480,NaN,NaN,NaN,NaN,NaN,NaN
1,2017-01-05,0,675.0,0.001483,-0.001483,0.007435,0.005952,2.5295,-0.007287,NaN,NaN,NaN,NaN,NaN
2,2017-01-06,0,668.0,-0.010425,0.001483,-0.001483,0.000000,2.5380,0.003355,-0.007287,NaN,NaN,NaN,NaN
3,2017-01-07,1,668.0,0.000000,-0.010425,0.001483,-0.008942,2.5380,0.000000,0.003355,-0.007287,NaN,NaN,-0.003932
4,2017-01-08,1,668.0,0.000000,0.000000,-0.010425,-0.010425,2.5380,0.000000,0.000000,0.003355,-0.007287,NaN,0.003355
5,2017-01-09,0,668.0,0.000000,0.000000,0.000000,0.000000,2.5300,-0.003157,0.000000,0.000000,0.003355,-0.007287,0.000000
6,2017-01-10,0,668.0,0.000000,0.000000,0.000000,0.000000,2.6035,0.028637,-0.003157,0.000000,0.000000,0.003355,-0.003157
7,2017-01-11,0,670.0,0.002990,0.000000,0.000000,0.000000,2.6010,-0.000961,0.028637,-0.003157,0.000000,0.000000,0.025480
8,2017-01-12,0,678.0,0.011870,0.002990,0.000000,0.002990,2.6605,0.022618,-0.000961,0.028637,-0.003157,0.000000,0.027677
9,2017-01-13,0,672.0,-0.008889,0.011870,0.002990,0.014859,2.6805,0.007489,0.022618,-0.000961,0.028637,-0.003157,0.021657


In [345]:
df.shape

(2850, 14)

In [346]:
df.dropna(inplace=True)
df.shape

(2845, 14)

In [348]:
df.corr()[["y"]].sort_values(by="y", ascending=False)

,y
y,1.000000
usd_lag_1,0.178329
usd_lag_sum_1_2,0.098765
iata_adj,0.029990
dates,0.008064
copper_close,-0.009120
dummy_non_bussiness_day,-0.018478
usd_lag_2,-0.026711
copper_diff,-0.044706
copper_lag_4,-0.086968


## In-Sample Analysis

In [349]:
import statsmodels.api as sm

In [350]:
X = df[["usd_lag_1", "usd_lag_2"]]
y = df["y"]

X = sm.add_constant(X, prepend=True)
reg = sm.OLS(y, X)
reg = reg.fit()

reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.035
Model:                            OLS   Adj. R-squared:                  0.035
Method:                 Least Squares   F-statistic:                     52.05
Date:                Sun, 19 Jan 2025   Prob (F-statistic):           6.27e-23
Time:                        12:13:25   Log-Likelihood:                 10451.
No. Observations:                2845   AIC:                        -2.090e+04
Df Residuals:                    2842   BIC:                        -2.088e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0001      0.000      0.937      0.349      -0.000       0.000
usd_lag_1      0.1891      0.019     10.100      0.000       0.152       0.226
usd_lag_2     -0.0604      0.019     -3.228      0.001      -0.097      -0.024
==============================================================================
Omnibus:                      584.645   Durbin-Watson:                   1.998
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            14831.530
Skew:                          -0.320   Prob(JB):                         0.00
Kurtosis:                      14.167   Cond. No.                         176.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [351]:
np.sqrt(np.mean(reg.resid ** 2))

np.float64(0.006142886832096238)

In [356]:
X = df[["usd_lag_1", "usd_lag_2", "dummy_non_bussiness_day"]]
y = df["y"]

X = sm.add_constant(X, prepend=True)
reg = sm.OLS(y, X)
reg = reg.fit()

reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.036
Model:                            OLS   Adj. R-squared:                  0.035
Method:                 Least Squares   F-statistic:                     34.97
Date:                Sun, 19 Jan 2025   Prob (F-statistic):           3.45e-22
Time:                        12:13:57   Log-Likelihood:                 10452.
No. Observations:                2845   AIC:                        -2.090e+04
Df Residuals:                    2841   BIC:                        -2.087e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
===========================================================================================
                              coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------
const                       0.0002      0.000      1.275      0.202   -9.41e-05       0.000
usd_lag_1                   0.1887      0.019     10.077      0.000       0.152       0.225
usd_lag_2                  -0.0617      0.019     -3.286      0.001      -0.099      -0.025
dummy_non_bussiness_day    -0.0002      0.000     -0.899      0.369      -0.001       0.000
==============================================================================
Omnibus:                      593.409   Durbin-Watson:                   1.996
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            14886.463
Skew:                          -0.349   Prob(JB):                         0.00
Kurtosis:                      14.184   Cond. No.                         186.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [357]:
np.sqrt(np.mean(reg.resid ** 2))

np.float64(0.006142012783328181)

In [358]:
X = df[["usd_lag_1", "usd_lag_2", "copper_lag_1", "copper_lag_2", ]]
y = df["y"]

X = sm.add_constant(X, prepend=True)
reg = sm.OLS(y, X)
reg = reg.fit()

reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.072
Model:                            OLS   Adj. R-squared:                  0.071
Method:                 Least Squares   F-statistic:                     54.96
Date:                Sun, 19 Jan 2025   Prob (F-statistic):           1.08e-44
Time:                        12:14:08   Log-Likelihood:                 10506.
No. Observations:                2845   AIC:                        -2.100e+04
Df Residuals:                    2840   BIC:                        -2.097e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const            0.0001      0.000      1.226      0.220   -8.32e-05       0.000
usd_lag_1        0.1633      0.019      8.764      0.000       0.127       0.200
usd_lag_2       -0.0594      0.018     -3.232      0.001      -0.095      -0.023
copper_lag_1    -0.0861      0.010     -8.830      0.000      -0.105      -0.067
copper_lag_2    -0.0601      0.010     -6.088      0.000      -0.079      -0.041
==============================================================================
Omnibus:                      636.831   Durbin-Watson:                   2.024
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            19157.099
Skew:                          -0.371   Prob(JB):                         0.00
Kurtosis:                      15.691   Cond. No.                         178.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [359]:
np.sqrt(np.mean(reg.resid ** 2))

np.float64(0.0060255208698616666)

In [360]:
X = df[["usd_lag_sum_1_2", "copper_lag_sum_1_2"]]
y = df["y"]

X = sm.add_constant(X, prepend=True)
reg = sm.OLS(y, X)
reg = reg.fit()

reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.051
Model:                            OLS   Adj. R-squared:                  0.050
Method:                 Least Squares   F-statistic:                     76.12
Date:                Sun, 19 Jan 2025   Prob (F-statistic):           6.26e-33
Time:                        12:14:17   Log-Likelihood:                 10474.
No. Observations:                2845   AIC:                        -2.094e+04
Df Residuals:                    2842   BIC:                        -2.092e+04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                  0.0001      0.000      1.237      0.216   -8.28e-05       0.000
usd_lag_sum_1_2        0.0491      0.012      4.098      0.000       0.026       0.073
copper_lag_sum_1_2    -0.0787      0.007    -11.092      0.000      -0.093      -0.065
==============================================================================
Omnibus:                      767.594   Durbin-Watson:                   1.828
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            27323.908
Skew:                          -0.584   Prob(JB):                         0.00
Kurtosis:                      18.137   Cond. No.                         105.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [361]:
np.sqrt(np.mean(reg.resid ** 2))

np.float64(0.006093314102755903)

In [365]:
X = df[["usd_lag_1", "usd_lag_2", "copper_lag_1", "copper_lag_2", "copper_lag_3", "copper_lag_4"]]
y = df["y"]

X = sm.add_constant(X, prepend=True)
reg = sm.OLS(y, X)
reg = reg.fit()

reg.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.091
Model:                            OLS   Adj. R-squared:                  0.090
Method:                 Least Squares   F-statistic:                     47.61
Date:                Sun, 19 Jan 2025   Prob (F-statistic):           6.70e-56
Time:                        12:14:28   Log-Likelihood:                 10536.
No. Observations:                2845   AIC:                        -2.106e+04
Df Residuals:                    2838   BIC:                        -2.102e+04
Df Model:                           6                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const            0.0002      0.000      1.490      0.136   -5.28e-05       0.000
usd_lag_1        0.1409      0.019      7.547      0.000       0.104       0.178
usd_lag_2       -0.0868      0.019     -4.685      0.000      -0.123      -0.050
copper_lag_1    -0.0873      0.010     -9.042      0.000      -0.106      -0.068
copper_lag_2    -0.0650      0.010     -6.633      0.000      -0.084      -0.046
copper_lag_3    -0.0642      0.010     -6.511      0.000      -0.083      -0.045
copper_lag_4    -0.0462      0.010     -4.724      0.000      -0.065      -0.027
==============================================================================
Omnibus:                      640.298   Durbin-Watson:                   2.007
Prob(Omnibus):                  0.000   Jarque-Bera (JB):            19461.396
Skew:                          -0.375   Prob(JB):                         0.00
Kurtosis:                      15.791   Cond. No.                         178.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [366]:
np.sqrt(np.mean(reg.resid ** 2))

np.float64(0.005961533828063382)